# About This Notebook 

This notebook was done on Kaggle and downloaded after. It contains code that we use to scrape the games data from the 30k to 40k rows of our list of games. 

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/steam60kgamesdata/gamesdata.csv


In [2]:
df = pd.read_csv('/kaggle/input/steam60kgamesdata/gamesdata.csv')

In [3]:
from bs4 import BeautifulSoup
import time
import requests
import json
from tqdm import tqdm

In [4]:
counter = 0
data = []

for counter in tqdm(range(30000,40000)):
        hlink = df['Link'][counter]
        response = requests.get(hlink)
        soup1 = BeautifulSoup(response.text, 'html.parser')

        description = soup1.find('div',id='game_area_description')
        description = description.text.strip().replace('\r','').replace('\t','').replace('\n','') if description else None

        reviews = soup1.find('div', class_='user_reviews_summary_row')
        reviews = reviews.text.strip().replace('\r','').replace('\t','').replace('\n','') if reviews else None
        tags_div = soup1.find('div', class_='glance_tags popular_tags')
        tags = tags_div.find_all('a', class_='app_tag') if tags_div else 'N/A'

        if tags != 'N/A':
            tagged = [tag.text.strip() for tag in tags]
            tagged_string = ', '.join(tagged)
        else:
            tagged_string = 'N/A'

       
        data.append([description, reviews, tagged_string])
        
        counter+=1
    

100%|██████████| 10000/10000 [2:45:02<00:00,  1.01it/s]


In [5]:
data_df = pd.DataFrame(data, columns=['description','reviews','tags'])
data_df

,description,reviews,tags
0,About This GameWarPlan is a World War II simul...,All Reviews:Mostly Positive(104)- 77% of the 1...,"Strategy, Simulation, Wargame, Turn-Based Stra..."
1,"About This GameBloodlust 2: Nemesis is a ""Hack...",All Reviews:Mostly Positive(88)- 75% of the 88...,"RPG, Action, Indie, Vampire, Hack and Slash, D..."
2,About This Game-----Include---------------1 bi...,All Reviews:7 user reviews- Need more user rev...,"Action, Indie, Casual, Racing, Fast-Paced"
3,About This GameWelcome to the magic city of pa...,All Reviews:Mostly Positive(99)- 79% of the 99...,"Adventure, Casual, Hidden Object, 2D, Detectiv..."
4,About This GameTip: Please enter the number “8...,All Reviews:Mostly Positive(38)- 71% of the 38...,"Indie, Casual, Sexual Content, Nudity, Mature,..."
...,...,...,...
9995,"About This GameIn Witch of the Woods, you play...",All Reviews:7 user reviews- Need more user rev...,"Indie, Strategy"
9996,About This GameSTOP PLAYING AS SOMEONE ELSE!Th...,All Reviews:2 user reviews- Need more user rev...,"Action, Indie, Adventure"
9997,About This GameSpellFront is a first person ac...,All Reviews:Mixed(39)- 61% of the 39 user revi...,"Early Access, Action, Indie, Violent, Adventur..."
9998,About This GameBeyond the Void is a new hybrid...,All Reviews:Mixed(124)- 58% of the 124 user re...,"Free to Play, Strategy, Indie, Multiplayer, Sp..."


In [6]:
data_df.to_csv('30to40_k_games.csv', index=False)